In [1]:
import pandas as pd
import numpy as np
import warnings

# enable warnings
warnings.filterwarnings('default')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import classification_report, precision_recall_curve

In [3]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Read the CSV file from the drive
df = pd.read_csv('/content/drive/MyDrive/Data Folder/Bookings.csv')

# Print the first few rows of the DataFrame
df.head()

Mounted at /content/drive


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-2015
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,02-07-2015
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,03-07-2015


In [4]:
df.duplicated().sum()

31994

In [5]:
duplicates = df[df.duplicated(keep=False)]
duplicates.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03-07-2015
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03-07-2015
21,Resort Hotel,0,72,2015,July,27,1,2,4,2,...,No Deposit,250.0,NaN,0,Transient,84.67,0,1,Check-Out,07-07-2015
22,Resort Hotel,0,72,2015,July,27,1,2,4,2,...,No Deposit,250.0,NaN,0,Transient,84.67,0,1,Check-Out,07-07-2015
39,Resort Hotel,0,70,2015,July,27,2,2,3,2,...,No Deposit,250.0,NaN,0,Transient,137.00,0,1,Check-Out,07-07-2015
43,Resort Hotel,0,70,2015,July,27,2,2,3,2,...,No Deposit,250.0,NaN,0,Transient,137.00,0,1,Check-Out,07-07-2015
132,Resort Hotel,1,5,2015,July,28,5,1,0,2,...,No Deposit,240.0,NaN,0,Transient,97.00,0,0,Canceled,01-07-2015
138,Resort Hotel,1,5,2015,July,28,5,1,0,2,...,No Deposit,240.0,NaN,0,Transient,97.00,0,0,Canceled,01-07-2015
198,Resort Hotel,0,0,2015,July,28,7,0,1,1,...,No Deposit,240.0,NaN,0,Transient,109.80,0,3,Check-Out,08-07-2015
200,Resort Hotel,0,0,2015,July,28,7,0,1,1,...,No Deposit,240.0,NaN,0,Transient,109.80,0,3,Check-Out,08-07-2015


In [6]:
# drop duplicates while keeping the first occurrence of each row
df = df.drop_duplicates(keep='first')

In [7]:
# Preprocessing the data
df['children'] = df['children'].fillna(0)
df['country'] = df['country'].fillna('Unknown')
df['agent'] = df['agent'].fillna(0)
df['company'] = df['company'].fillna(0)

In [8]:
df = df.drop(['reservation_status', 'reservation_status_date'], axis=1)
df['hotel'] = df['hotel'].replace({'Resort Hotel': 0, 'City Hotel': 1})

In [9]:
df = pd.get_dummies(df, columns=['meal', 'market_segment', 'arrival_date_month',
                                 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'country'])


In [10]:
# Splitting the data into training and testing sets
X = df.drop(['is_canceled'], axis=1)
y = df['is_canceled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
# Fitting a logistic regression model
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [13]:
# Predicting on the test set
y_pred = lr.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = lr.predict(X_test)
cm_lr = confusion_matrix(y_test, y_pred)
print(cm_lr)
print('-- --' * 15)
print(accuracy_score(y_test, y_pred))
print('-- --' * 15)
print(classification_report(y_test, y_pred))
print('-- --' * 15)

[[11626  1107]
 [ 2432  2315]]
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
0.7975400457665904
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
              precision    recall  f1-score   support

           0       0.83      0.91      0.87     12733
           1       0.68      0.49      0.57      4747

    accuracy                           0.80     17480
   macro avg       0.75      0.70      0.72     17480
weighted avg       0.79      0.80      0.79     17480

-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --


In [15]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
model2.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model2.predict(X_test)
cm_dt = confusion_matrix(y_test, y_pred)
print(cm_dt)
print('-- --' * 15)
print(accuracy_score(y_test, y_pred))
print('-- --' * 15)
print(classification_report(y_test, y_pred))
print('-- --' * 15)

[[10987  1746]
 [ 1652  3095]]
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
0.8056064073226544
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     12733
           1       0.64      0.65      0.65      4747

    accuracy                           0.81     17480
   macro avg       0.75      0.76      0.76     17480
weighted avg       0.81      0.81      0.81     17480

-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --


In [17]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
model3.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred1 = model3.predict(X_test)
cm_rf = confusion_matrix(y_test, y_pred1)
print(cm_rf)
print('-- --' * 15)
print(accuracy_score(y_test, y_pred1))
print('-- --' * 15)
print(classification_report(y_test, y_pred1))
print('-- --' * 15)

[[11883   850]
 [ 1992  2755]]
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
0.8374141876430206
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
              precision    recall  f1-score   support

           0       0.86      0.93      0.89     12733
           1       0.76      0.58      0.66      4747

    accuracy                           0.84     17480
   macro avg       0.81      0.76      0.78     17480
weighted avg       0.83      0.84      0.83     17480

-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --


In [19]:
from sklearn.svm import SVC
model4 = SVC(kernel = 'linear', random_state = 0)
model4.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred2 = model4.predict(X_test)
cm = confusion_matrix(y_test, y_pred2)
print(cm)
print('-- --' * 15)
print(accuracy_score(y_test, y_pred2))
print('-- --' * 15)
print(classification_report(y_test, y_pred2))
print('-- --' * 15)

[[11758   975]
 [ 2511  2236]]
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
0.8005720823798627
-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --
              precision    recall  f1-score   support

           0       0.82      0.92      0.87     12733
           1       0.70      0.47      0.56      4747

    accuracy                           0.80     17480
   macro avg       0.76      0.70      0.72     17480
weighted avg       0.79      0.80      0.79     17480

-- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- ---- --


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

# define the Random Forest Classifier
rfc = RandomForestClassifier()

# define the range of hyperparameters to search over
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [None, 5, 10],
              'max_features': ['sqrt', 'log2']}

# define the GridSearchCV object with 5-fold cross-validation
grid_search = GridSearchCV(rfc, param_grid, cv=5)

# fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# get the best hyperparameters
best_params = grid_search.best_params_

# retrain the model using the best hyperparameters
rfc_best = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                                   max_depth=best_params['max_depth'],
                                   max_features=best_params['max_features'])
rfc_best.fit(X_train, y_train)

# make predictions on the test data
y_pred = rfc_best.predict(X_test)

# calculate the evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8514302059496568
Precision: 0.7831928345626976
Recall: 0.6262902886033285
